# Imports

In [1]:
import os

import torch
import numpy as np
from torch import optim, nn
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from mask_detector.dataset import generate_train_datasets, DatasetType
from mask_detector.models import GenderClassifierModel
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import pytz

print(f"PyTorch version: {torch.__version__}.")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"device: {device}")
print(f"device count: {torch.cuda.device_count()}")

PyTorch version: 1.6.0.
device: cuda:0
device count: 1


# Hyperparameters and Settings

## Settings

In [2]:
save_path = "./result/"
dataset_root = "/opt/ml/input/data"
if not os.path.isdir(save_path):
    os.mkdir(save_path)

if not os.path.isdir(save_path + "/tensorboard"):
    os.mkdir(save_path + "/tensorboard")

if not os.path.isdir(save_path + "/checkpoint"):
    os.mkdir(save_path + "/checkpoint")


In [3]:
use_cuda = "cuda" in str(device)
load_prev_model = False

log_batch_count = 30

## Hyperparameters

In [4]:
random_seed = 1004
batch_size = 256

epochs = 128
lr = 0.001

# 최대 Iteration 횟수
t_max = 30
# 최소로 떨어질 수 있는 값
eta_min = 0

## Functions and Tools
### Tensorboard

In [5]:
kst = pytz.timezone('Asia/Seoul')
logger = SummaryWriter(log_dir=save_path + "/tensorboard/" + datetime.now(kst).strftime("%Y-%m-%d %H:%M:%S"))

### Others

In [6]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [7]:
def make_sample(images, labels, predicts):
    if images.shape[0] > 8:
        images = images[0:8]
        labels = labels[0:8]
        predicts = predicts[0:8]

    sample_figure = plt.figure(figsize=(20, 10))
    label_dict = ["Male", "Female"]

    for idx, (image, label, predict) in enumerate(zip(images, labels.squeeze(), predicts)):
        label = label.item()
        predict = predict.item()

        plt.subplot(2, 4, idx + 1, title=f"Pred: {label_dict[predict]} / Label: {label_dict[label]}")
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image)
    
    return sample_figure

# Generate Dataset

In [8]:
train_set, valid_set = generate_train_datasets(dataset_root)

100%|██████████| 676/676 [00:14<00:00, 46.03it/s]


In [9]:
print("---- Switch training dataset to gender dataset")
train_set.generate_serve_list(DatasetType.Gender, shuffle=True)
print("---- Switch to validatation dataset as gender dataset")
valid_set.generate_serve_list(DatasetType.Gender, shuffle=True)

print()
print(f"Train Set Size: {len(train_set)}")
print(f"Valid Set Size: {len(valid_set)}")

---- Switch training dataset to gender dataset
-- Original Data
Lesser Data: 5782
Greater Data: 8386

-- Oversampled Data
Lesser Data: 8386
Greater Data: 8386

---- Switch to validatation dataset as gender dataset
-- Original Data
Lesser Data: 1512
Greater Data: 3220

-- Oversampled Data
Lesser Data: 3220
Greater Data: 3220


Train Set Size: 16772
Valid Set Size: 6440


In [10]:
train_set_loader = DataLoader(train_set, batch_size, num_workers=4, shuffle=True, pin_memory=use_cuda)
valid_set_loader = DataLoader(valid_set, batch_size, num_workers=4, shuffle=True, pin_memory=use_cuda)

print("Generate Dataset Complete")

Generate Dataset Complete


In [11]:
# sample = None
# for epoch in range(epochs):
#     for idx, (sources, labels, path_texts) in enumerate(train_set_loader):
#         sources = sources.to(device)
#         labels = labels.to(device)

#         outputs = model(sources)
#         predicts = torch.argmax(outputs, dim=-1)

#         source_new = torch.clone(sources).detach().cpu().permute(0, 2, 3, 1).numpy()
#         sample = make_sample(source_new, labels, predicts)
        
#         break
#     break

# sample.show()


# Model, Loss and Metric Define

In [12]:
model = GenderClassifierModel().to(device)
model = torch.nn.DataParallel(model)    # GPU는 하나 밖에 없는데....? ㅠㅠ

if load_prev_model:
    model.load_state_dict(torch.load(f"{save_path}/gender_last_model.pth"))


Loaded pretrained weights for efficientnet-b0


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=t_max, eta_min=eta_min)

# Training

In [14]:
best_valid_accuracy = 0
best_valid_loss = np.inf
for epoch in range(epochs):
    # Train loop
    print(f"\n----- Start Epoch: {datetime.now()}-----\n")

    model.train()
    loss_value = 0
    matches = 0
    for idx, (sources, labels, path_texts) in enumerate(train_set_loader):
        # Load Data
        sources = sources.to(device)
        labels = labels.to(device, dtype=torch.long)

        # Initialize Gradient
        optimizer.zero_grad()

        # Forward
        outputs = model(sources)
        predicts = torch.argmax(outputs, dim=-1)
        loss = criterion(outputs, labels)
        
        # Backpropagation
        loss.backward()
        optimizer.step()

        # Examination
        loss_value += loss.item()
        matches += (predicts == labels).sum().item()
        if (idx + 1) % log_batch_count == 0:
            # Calc
            train_loss = loss_value / log_batch_count
            train_accuracy = matches / (batch_size * log_batch_count)
            current_lr = get_lr(optimizer)

            # Print examination result
            print(f"Epoch: [{epoch + 1}/{epochs}] ({idx + 1}/{len(train_set_loader)})")
            print(f"Training loss: {train_loss:4.4}")
            print(f"Training accuracy: {train_accuracy:4.2%}")
            print(f"Learning Rate: {current_lr}")
            print()

            # Write at Tensorboard
            logger.add_scalar("Train/loss", train_loss, epoch * len(train_set_loader) + idx)
            logger.add_scalar("Train/accuracy", train_accuracy, epoch * len(train_set_loader) + idx)

            # Initialize examination
            loss_value = 0
            matches = 0

    scheduler.step()

    # Validation
    with torch.no_grad():    
        print("Calculating validation results...")
        model.eval()

        loss_value = 0
        matches = 0
        valid_sample = None
        for sources, labels, path_texts in valid_set_loader:
            sources = sources.to(device)
            labels = labels.to(device, dtype=torch.long)

            outputs = model(sources)
            predicts = torch.argmax(outputs, dim=-1)
            loss = criterion(outputs, labels)
            loss_value += loss.item()
            matches += (predicts == labels).sum().item()

            # 추후 Label Smotheness 적용 후 가장 안 좋은 결과에 대해 저장
            if valid_sample is None:
                source_new = torch.clone(sources).detach().cpu().permute(0, 2, 3, 1).numpy()
                valid_sample = make_sample(source_new, labels, predicts)
        
        # Validation 결과 계산
        valid_loss = loss_value / len(valid_set_loader)
        valid_accuracy = matches / len(valid_set)
        best_valid_loss = min(best_valid_loss, valid_loss)

        # 모델 저장
        # 추후 f1-score로 바꾸기
        if valid_accuracy > best_valid_accuracy:
            best_valid_accuracy = valid_accuracy
            print(f"New best model for val accuracy : {valid_accuracy:4.2%}! saving the best model..")
            torch.save(model.state_dict(), f"{save_path}/checkpoint/gender_best_model.pth")
        torch.save(model.state_dict(), f"{save_path}/checkpoint/gender_last_model.pth")
        
        # Validation 결과 출력
        print(f"Best loss: {best_valid_loss:4.4}")
        print(f"Current loss: {valid_loss:4.4}")
        print(f"Best accuracy: {best_valid_accuracy:4.2%}")
        print(f"Current accuracy: {valid_accuracy:4.2%}")

        logger.add_scalar("Val/loss", valid_loss, epoch)
        logger.add_scalar("Val/accuracy", valid_accuracy, epoch)
        logger.add_figure("Results", valid_sample, epoch)

        print(f"\n----- End Epoch: {datetime.now()}-----\n")
            

cy: 92.55%
Current accuracy: 90.25%

----- End Epoch: 2021-04-04 18:45:35.383534-----


----- Start Epoch: 2021-04-04 18:45:35.388838-----

Epoch: [114/128] (30/66)
Training loss: 0.06602
Training accuracy: 97.85%
Learning Rate: 0.0008715724127386964

Epoch: [114/128] (60/66)
Training loss: 0.08434
Training accuracy: 97.20%
Learning Rate: 0.0008715724127386964

Calculating validation results...
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow wi